<a href="https://colab.research.google.com/github/NihPat95/Instacart-Market-Analysis/blob/master/Instacart_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import time

colors = sns.color_palette()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install efficient_apriori

In [0]:
path = '/content/drive/My Drive/Instacart/'
orders = pd.read_pickle(path + 'customers_orders10000.pkl')

In [0]:
products = pd.read_csv(path + 'products.csv')
productIdMap = products.set_index(['product_id'])['product_name'].to_dict()

In [0]:
from efficient_apriori import apriori
itemsets, rules = apriori(orders['items'], min_support=0.01,  min_confidence=0.01)

In [0]:
def itemsets_id_to_name(itemsets):
  itemsets_names = {}
  for k, v in itemsets.items():
    itemsets_names[k] = {}
    for k2, v2 in v.items():
      itemsets_names[k][tuple([productIdMap[i] for i in k2])] = v2
  return itemsets_names

In [0]:
from efficient_apriori import Rule
def rules_id_to_name(rules):
  rules_name = [Rule(tuple([productIdMap[i] for i in rule.lhs]), 
                   tuple([productIdMap[i] for i in rule.rhs]),
                   rule.count_full, rule.count_lhs, 
                   rule.count_rhs, rule.num_transactions) for rule in rules]
  return rules_name

In [65]:
itemsets_id_to_name(itemsets)

{1: {('3D White Radiant Mint Flavor Whitening Toothpaste',): 72,
  ('93% Ground Beef',): 379,
  ('Aged Red Wine Vinegar',): 106,
  ('Baby Food Pears Squash',): 169,
  ('Balsamic Vinegar',): 64,
  ('Butter Croissant',): 74,
  ('Cannellini White Kidney Beans No Salt Added',): 365,
  ('Carb Balance Small Fajita Whole Wheat Tortillas',): 92,
  ('Chewy Peanut Peanut Butter Granola Bars',): 83,
  ('Chicken Quesadilla',): 62,
  ('Chocolate Peanut Butter',): 314,
  ('Choose-A-Sheet Regular Roll Paper Towels',): 207,
  ('Chopped Spinach Steamers',): 78,
  ('Classic Mixed Vegetables',): 94,
  ('Cranberries',): 99,
  ('Dairy Free Hazelnut Coconut Milk Creamer',): 183,
  ('Dark Chocolate Covered Goji Berries',): 71,
  ('European-Style Cultured Butter',): 79,
  ('Fresh Cut Green Beans',): 171,
  ('Freshly made pasta layered between a rich meat sauce and topped with Real Mozzarella cheese Lasagna with Meat & Sauce',): 78,
  ('Gluten Free Chocolate Coated Wafers',): 125,
  ('Limeade',): 190,
  ('Orga

In [66]:
rules_id_to_name(rules)

[{Organic Frozen Peas} -> {93% Ground Beef},
 {93% Ground Beef} -> {Organic Frozen Peas},
 {Pureed Baby Food Strawberries Carrots} -> {Baby Food Pears Squash},
 {Baby Food Pears Squash} -> {Pureed Baby Food Strawberries Carrots},
 {Organic Frozen Peas} -> {Organic Broccoli Crown},
 {Organic Broccoli Crown} -> {Organic Frozen Peas},
 {Soy Free Buttery Spread} -> {Organic Frozen Peas},
 {Organic Frozen Peas} -> {Soy Free Buttery Spread},
 {Cannellini White Kidney Beans No Salt Added} -> {Organic Frozen Peas},
 {Organic Frozen Peas} -> {Cannellini White Kidney Beans No Salt Added}]